# Tips from GCP training

In [1]:
# %%writefile "path"/"script_name".py - create a script file or other file with the cell content

In [ ]:
# %%bash - commnad line in the cell

## Imports

In [110]:
import numpy as np
import re
import pandas as pd
from config import config
from get_offers_by_city import connect

## Create Connection to de_rent Database

In [111]:
conn = connect(config())

Connecting to database...
Connection successful


## Load data from database

In [112]:
query = '''
    SELECT * FROM all_offers_raw_infos
'''

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchall()

In [113]:
df = pd.DataFrame(result, columns=['offer_id', 'extraction_date', 'city', 'city_code', 'offer_type', 'lat_lng', 'offer_infos'])

In [114]:
df.head()

,offer_id,extraction_date,city,city_code,offer_type,lat_lng,offer_infos
0,45678593,2021-11-03 05:14:31,Dresden,100051,wohnung,"['{lat: 50.10168,lng: 8.65827}']","['', '""area"":65,""mobex"":true,""zip"":""60327"",""ob..."
1,45776903,2021-11-03 05:14:31,Dresden,100051,wohnung,"['{lat: 51.04843,lng: 13.75065}']","['', '""area"":40,""mobex"":true,""zip"":""01069"",""ob..."
2,44924940,2021-11-03 05:14:31,Dresden,100051,wohnung,"['{lat: 51.04431325059276,lng: 13.699018108643...","['', '""area"":55,""mobex"":true,""zip"":""01159"",""ob..."
3,44630030,2021-11-03 05:14:31,Dresden,100051,wohnung,"['{lat: 50.11131,lng: 8.70392}']","['', '""area"":50,""mobex"":true,""zip"":""60314"",""ob..."
4,45514780,2021-11-03 05:14:31,Dresden,100051,wohnung,"['{lat: 51.49915,lng: 7.45147}']","['', '""area"":85.7,""mobex"":true,""zip"":""44139"",""..."


## Data Cleaning

### Cleaning one offer

#### Lat - Lng

In [6]:
# look to the information
df['lat_lng'][0]

"['{lat: 50.10168,lng: 8.65827}']"

In [7]:
# Separate into latitude (lat) and longitude (lng)
df['lat'] = df['lat_lng'].apply(lambda x: re.findall('\d+.\d+', x)[0])
df['lng'] = df['lat_lng'].apply(lambda x: re.findall('\d+.\d+', x)[1])

In [8]:
# drop original lat_lng column
df.drop(columns='lat_lng', inplace=True)

#### Separate infos from offer_infos

In [115]:
# Get all data in "JASON" format for each offer and put it into a dictionary
# and than into a list to create a now separated dataframe with the main informations.

# Separate into latitude (lat) and longitude (lng)
df['lat'] = df['lat_lng'].apply(lambda x: re.findall('\d+.\d+', x)[0])
df['lng'] = df['lat_lng'].apply(lambda x: re.findall('\d+.\d+', x)[1])

# drop original lat_lng column
df.drop(columns='lat_lng', inplace=True)

df_list = []

for x in range(len(df)):
    infos_dict = {}  
    
    infos_dict['offer_id'] = df['offer_id'][x]
    infos_dict['extraction_date'] = pd.to_datetime(df['extraction_date'][x])
    infos_dict['lat'] = df['lat'][x]
    infos_dict['lng'] = df['lng'][x]
    
    
    b = df['offer_infos'][x].replace('\\', '')
    b = b.replace('{', '').replace('}', '')[4:]
    b = b[:-1]
    b = b.split(',')
    for i in b:
        # offer area
        if 'area' in i:
            try:
                i = i.replace('"', '').replace("'", "").replace('area:', '').replace(' ', '')
                infos_dict['area_m2'] = float(i)
            except:
                infos_dict['area_m2'] = np.nan
        # if the offer is furnished or not
        if 'mobex' in i:
            if 'true' in i:
                infos_dict['furnished'] = 1
            elif 'false' in i:
                infos_dict['furnished'] = 0
            else:
                infos_dict['furnished'] = np.nan
        # the offer zip code 
        if 'zip' in i:
            try:
                infos_dict['zip_code'] = int(re.findall('\d+', i)[0])
            except:
                infos_dict['zip_code'] = np.nan
        # offer category
        if 'objectcat' in i:
            try:
                infos_dict['objectcat'] = re.findall('\:"\w+"', b[3])[0][1:].replace('"', '')
            except:
                infos_dict['objectcat'] = np.nan
        # number of rooms
        if 'rooms' in i:
            try:
                infos_dict['rooms'] = float(re.findall('\d+', i)[0])
            except:
                infos_dict['rooms'] = np.nan
        # build yuear of construction
        if 'buildyear' in i:
            try:
                infos_dict['build_year'] = int(re.findall('\d+', i)[0])
            except:
                infos_dict['build_year'] = np.nan
        # state
        if 'fed' in i:
            try:
                infos_dict['state'] = i.split(':')[1].replace('"', '')
            except:
                infos_dict['state'] = np.nan
        # city
        if 'city' in i:
            try:
                infos_dict['city'] = i.split(':')[1].replace('"', '')
            except:
                infos_dict['city'] = np.nan
        # offer sub-category
        if 'obcat' in i:
            try:
                infos_dict['sub_category'] = i.split(':')[1].replace('"', '')
            except:
                infos_dict['sub_category'] = np.nan
        # if the offer has or not a "balcon"- balcony
        if 'balcn' in i:
            if 'true' in i:
                infos_dict['balcony'] = 1
            elif 'false' in i:
                infos_dict['balcony'] = 0
            else:
                infos_dict['balcony'] = np.nan
        # heat type
        if 'heatr' in i:
            try:
                infos_dict['heat_type'] = i.split(':')[1].replace('"', '')
            except:
                infos_dict['heat_type'] = np.nan
        # offer title
        if 'title' in i:
            try:
                infos_dict['offer_title'] = i.split(':')[1].replace('"', '')
            except:
                infos_dict['offer_title'] = np.nan
        # if the offer has already a kitchen
        if 'kitch' in i:
            if 'true' in i:
                infos_dict['kitchen'] = 1
            elif 'false' in i:
                infos_dict['kitchen'] = 0
            else:
                infos_dict['kitchen'] = np.nan
        # if the offer has a garden or not
        if 'gardn' in i:
            if 'true' in i:
                infos_dict['garden'] = 1
            elif 'false' in i:
                infos_dict['garden'] = 0
            else:
                infos_dict['garden'] = np.nan
        # offer rent price
        if 'price' in i:
            try:
                infos_dict['rent_price'] = float(re.findall('\d+', i)[0])
            except:
                infos_dict['rent_price'] = np.nan
    df_list.append(infos_dict)

df_cleaned = pd.DataFrame(df_list)
       

In [108]:
df_cleaned.columns

Index(['offer_id', 'extraction_date', 'lat', 'lng', 'area_m2', 'furnished',
       'zip_code', 'objectcat', 'rooms', 'build_year', 'state', 'city',
       'sub_category', 'balcony', 'heat_type', 'offer_title', 'kitchen',
       'rent_price', 'garden'],
      dtype='object')

In [117]:
df_cleaned.dtypes

offer_id                    int64
extraction_date    datetime64[ns]
lat                        object
lng                        object
area_m2                   float64
furnished                   int64
zip_code                  float64
objectcat                  object
rooms                     float64
build_year                float64
state                      object
city                       object
sub_category               object
balcony                     int64
heat_type                  object
offer_title                object
kitchen                     int64
rent_price                float64
garden                      int64
dtype: object

In [116]:
df_cleaned.head()

,offer_id,extraction_date,lat,lng,area_m2,furnished,zip_code,objectcat,rooms,build_year,state,city,sub_category,balcony,heat_type,offer_title,kitchen,rent_price,garden
0,45678593,2021-11-03 05:14:31,50.10168,8.65827,65.0,1,60327.0,Wohnung,3.0,1903.0,Hessen,Frankfurt am Main,Etagenwohnung,0,Zentralheizung,* frisch renovierte 3 Zimmerwohnung * Innensta...,0,790.0,0
1,45776903,2021-11-03 05:14:31,51.04843,13.75065,40.0,1,1069.0,Wohnung,2.0,2021.0,Sachsen,Dresden,Etagenwohnung,0,NaN,Zwischen Elbe und Großem Garten - Neubau 2-Zim...,0,420.0,0
2,44924940,2021-11-03 05:14:31,51.04431325059276,13.69901810864305,55.0,1,1159.0,Wohnung,2.0,NaN,Sachsen,Dresden,Etagenwohnung,0,Zentralheizung,Frisch sanierte 2 Raum Wohnung in ruhiger Lage...,0,440.0,0
3,44630030,2021-11-03 05:14:31,50.11131,8.70392,50.0,1,60314.0,Wohnung,2.0,1955.0,Hessen,Frankfurt am Main,Wohnung,0,Zentralheizung,wunderschön möbilierte Wohnung gegenüber ECB,1,1299.0,0
4,45514780,2021-11-03 05:14:31,51.49915,7.45147,85.7,1,44139.0,Wohnung,3.0,2021.0,Nordrhein-Westfalen,Dortmund,Etagenwohnung,1,NaN,Erstaunlich gut wohnen! VIVAWEST lässt Ihnen d...,0,935.0,0


In [104]:
df_cleaned.shape

(6549, 19)

In [105]:
df_cleaned.isna().sum()

offer_id              0
extraction_date       0
lat                   0
lng                   0
area_m2             487
furnished             0
zip_code              1
objectcat             0
rooms                65
build_year         1341
state                 0
city                  2
sub_category          0
balcony               0
heat_type          3019
offer_title           0
kitchen               0
rent_price           19
garden                0
dtype: int64

## Data Cleaning script

In [118]:
%%writefile '../scripts/offers_infos_cleaner.py'

'''Get the raw data from the DB, clean and organize it.

This script gets the raw dataset with all rent offers in the predefinated cities
in Germany, separate the meaningful information, clean it and organize it in 
different columns.

Returns a new dataframe read to be used.
'''
    
# imports
import re
import logging
import numpy as np
import pandas as pd
from config import config
from get_offers_by_city import connect

# set log folder, files and object configs
if not os.path.exists('Logs'):
    os.makedirs('Logs')
    
logging.basicConfig(
    filename='Logs/offers_infos_cleaner.txt',
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    datefmt='%Y-%m_%d %H:%M:%S',
    level=logging.DEBUG
)

logger = logging.getLogger('offers_infos_cleaner')


def get_data_from_db(conn):
'''Get the latest number os offers for each city.
    
    Parameters:
    ----------
        conn: connection to the database to extract needed infos
        
    Return:
    -------
        Return all offer raw infos to be cleaned.
        
    '''   
    
    # create a cursor object
    cursor = conn.cursor()
    
    # get infos from the database
    query = '''
        SELECT * FROM all_offers_raw_infos
    '''
    
    cursor.execute(query)
    result = cursor.fetchall()
    df_raw = pd.DataFrame(result, columns=['offer_id', 'extraction_date', 'city', 'city_code', 'offer_type', 'lat_lng', 'offer_infos'])
    
    return df_raw


def offers_infos_cleaning(df_raw):
    '''Clean and separate meaningful infos
    
    Parameter:
    ----------
        df_raw: Dataframe to be cleaned
        
    Return:
    -------
        Returns a new dataframe with the meaningful informations separated by columns
        and cleaned.    
    '''
    # Separate into latitude (lat) and longitude (lng)
    df['lat'] = df['lat_lng'].apply(lambda x: re.findall('\d+.\d+', x)[0])
    df['lng'] = df['lat_lng'].apply(lambda x: re.findall('\d+.\d+', x)[1])

    # drop original lat_lng column
    df.drop(columns='lat_lng', inplace=True)

    df_list = []

    for x in range(len(df)):
        infos_dict = {}  
        
        # get infos from df_raw
        infos_dict['offer_id'] = df['offer_id'][x]
        infos_dict['extraction_date'] = df['extraction_date'][x]
        infos_dict['lat'] = df['lat'][x]
        infos_dict['lng'] = df['lng'][x]

        # preprocess the infos cell
        b = df['offer_infos'][x].replace('\\', '')
        b = b.replace('{', '').replace('}', '')[4:]
        b = b[:-1]
        b = b.split(',')
        
        # get all meaningful infos and return it cleane and
        # separated by columns.
        for i in b:
            # offer area
            if 'area' in i:
                try:
                    i = i.replace('"', '').replace("'", "").replace('area:', '').replace(' ', '')
                    infos_dict['area_m2'] = float(i)
                except:
                    infos_dict['area_m2'] = np.nan
                    logger.debug(f'Offer {i} has no information about area.')
            # if the offer is furnished or not
            if 'mobex' in i:
                if 'true' in i:
                    infos_dict['furnished'] = 1
                elif 'false' in i:
                    infos_dict['furnished'] = 0
                else:
                    infos_dict['furnished'] = np.nan
                    logger.debug(f'Offer {i} has no information about furniture.')
            else:
                infos_dict['furnished'] = np.nan
                logger.debug(f'Offer {i} has no information about furniture.')
            # the offer zip code 
            if 'zip' in i:
                try:
                    infos_dict['zip_code'] = int(re.findall('\d+', i)[0])
                except:
                    infos_dict['zip_code'] = np.nan
                    logger.debug(f'Offer {i} has no information about zip_code.')
            # offer category
            if 'objectcat' in i:
                try:
                    infos_dict['main_category'] = re.findall('\:"\w+"', b[3])[0][1:].replace('"', '')
                except:
                    infos_dict['main_category'] = np.nan
                    logger.debug(f'Offer {i} has no information about main category.')
            # number of rooms
            if 'rooms' in i:
                try:
                    infos_dict['rooms'] = float(re.findall('\d+', i)[0])
                except:
                    infos_dict['rooms'] = np.nan
                    logger.debug(f'Offer {i} has no information about number of rooms.')
            # build yuear of construction
            if 'buildyear' in i:
                try:
                    infos_dict['build_year'] = int(re.findall('\d+', i)[0])
                except:
                    infos_dict['build_year'] = np.nan
                    logger.debug(f'Offer {i} has no information about build construction year.')
            # state
            if 'fed' in i:
                try:
                    infos_dict['state'] = i.split(':')[1].replace('"', '')
                except:
                    infos_dict['state'] = np.nan
                    logger.debug(f'Offer {i} has no information about state.')
            # city
            if 'city' in i:
                try:
                    infos_dict['city'] = i.split(':')[1].replace('"', '')
                except:
                    infos_dict['city'] = np.nan
                    logger.debug(f'Offer {i} has no information about city.')
            # offer sub-category
            if 'obcat' in i:
                try:
                    infos_dict['sub_category'] = i.split(':')[1].replace('"', '')
                except:
                    infos_dict['sub_category'] = np.nan
                    logger.debug(f'Offer {i} has no information about sub-category.')
            # if the offer has or not a "balcon"- balcony
            if 'balcn' in i:
                if 'true' in i:
                    infos_dict['balcony'] = 1
                elif 'false' in i:
                    infos_dict['balcony'] = 0
                else:
                    infos_dict['balcony'] = np.nan
                    logger.debug(f'Offer {i} has no information about balcony.')
            else:
                infos_dict['balcony'] = np.nan
                logger.debug(f'Offer {i} has no information about balcony.')
            # heat type
            if 'heatr' in i:
                try:
                    infos_dict['heat_type'] = i.split(':')[1].replace('"', '')
                except:
                    infos_dict['heat_type'] = np.nan
                    logger.debug(f'Offer {i} has no information about heat type.')
            # offer title
            if 'title' in i:
                try:
                    infos_dict['offer_title'] = i.split(':')[1].replace('"', '')
                except:
                    infos_dict['offer_title'] = np.nan
                    logger.debug(f'Offer {i} has no information about offer title.')
            # if the offer has already a kitchen
            if 'kitch' in i:
                if 'true' in i:
                    infos_dict['kitchen'] = 1
                elif 'false' in i:
                    infos_dict['kitchen'] = 0
                else:
                    infos_dict['kitchen'] = np.nan
                    logger.debug(f'Offer {i} has no information about kitchen.')
            else:
                infos_dict['kitchen'] = np.nan
                logger.debug(f'Offer {i} has no information about kitchen.')
            if 'gardn' in i:
                if 'true' in i:
                    infos_dict['garden'] = 1
                elif 'false' in i:
                    infos_dict['garden'] = 0
                else:
                    infos_dict['garden'] = np.nan
                    logger.debug(f'Offer {i} has no information about garden.')
            else:
                infos_dict['garden'] = np.nan
                logger.debug(f'Offer {i} has no information about garden.')
            # offer rent price
            if 'price' in i:
                try:
                    infos_dict['rent_price'] = float(re.findall('\d+', i)[0])
                except:
                    infos_dict['rent_price'] = np.nan
                    logger.debug(f'Offer {i} has no information rent price.')
                    
        # append the infos about the offer           
        df_list.append(infos_dict)
        logger.info(f'Offer no. {i} cleaned.')
        
    # create a new cleaned dataframe
    df_cleaned = pd.DataFrame(df_list)

    return df_cleaned

def load_df_cleaned(df_cleaned, conn):
    '''Get the informations and store in a database
    
    Params:
    -------
        df_infos: dataframe to be stored.
        conn: connection to the database.
    Return:
    -------
        None

    '''
    table_name = 'rent_infos_cleaned'
    # delete table 
    query1 = f'DROP TABLE IF EXISTS {table_name}'
    cursor = conn.cursor()
    print('initiated...')
    try:
        cursor.execute(query1)
        conn.commit()
        logger.info('Old table droped')
        print(f'Deleted {table_name} table.')
    except (Exception, psycopg2.DatabaseError) as error:
        logger.error(f"Error: {error}")
        print(f"Error: {error}")
        conn.rollback()
        cursor.close()
        return 1
    query2 = f'''CREATE TABLE IF NOT EXISTS {table_name} (        
        offer_id INTEGER, 
        extraction_date TIMESTAMP, 
        lat DOUBLE PRECISION, 
        lng DOUBLE PRECISION, 
        area_m2 REAL, 
        furnished BYTEA,
        zip_code INTEGER, 
        objectcat VASRCHAR(50), 
        rooms REAL, 
        build_year INTEGER, 
        state VARCHAR(50), 
        city VARCHAR(50),
        sub_category VARCHAR(50), 
        balcony BYTEA, 
        heat_type VARCHAR(50), 
        offer_title TEXT, 
        kitchen BYTEA,
        rent_price REAL, 
        garden BYTEA)''''
    try:
        cursor.execute(query2)
        conn.commit()
        logger.info('New table created')
        print('Recreated all_offer_ids table.')
    except (Exception, psycopg2.DatabaseError) as error:
        logger.error(f"Error: {error}")
        print(f"Error: {error}")
        conn.rollback()
        cursor.close()
        return 2
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df_infos.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df_infos.columns))
    # SQL quert to execute
    query3 = "INSERT INTO %s(%s) VALUES %%s" % (table_name, cols)
    try:
        extras.execute_values(cursor, query3, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        #logger.error(f"Error: {error}")
        conn.rollback()
        cursor.close()
        print(error)
        return 3
    logger.info(f"{table_name} uptodate.")
    print(f"{table_name} uptodate.")
    cursor.close()

def main():
    # connection to database
    conn = connect(config())
    
    df_raw = get_data_from_db(conn)
    df_cleaned = offers_infos_cleaning(df_raw)
    load_df_cleaned(df_cleaned, conn)
    
    conn.close()

if __name__=='__main__':
    main()

Writing ../scripts/offers_infos_cleaner.py
